In order to extract the audio clips from each episode, some of the data needs to be cleaned a little.

This is not an exploratory Notebook, because I made these mistakes while trying to put things together. Should have validated the data before hand, so, the following is just steps that I should have taken first

In [47]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import os
import re
#from fastai.vision import *
import random
from IPython.display import display
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', 1000)

## Episodes.csv

In [19]:
episodes = pd.read_csv("/notebooks/storage/ctds_data/Episodes.csv")
episodes.head()

,episode_id,episode_name,heroes,heroes_gender,heroes_location,heroes_nationality,heroes_kaggle_username,heroes_twitter_handle,category,flavour_of_tea,recording_date,recording_time,release_date,episode_duration,youtube_url,youtube_thumbnail_type,youtube_impressions,youtube_impression_views,youtube_ctr,youtube_nonimpression_views,youtube_views,youtube_watch_hours,youtube_avg_watch_duration,youtube_likes,youtube_dislikes,youtube_comments,youtube_subscribers,anchor_url,anchor_thumbnail_type,anchor_plays,spotify_starts,spotify_streams,spotify_listeners,apple_listeners,apple_listened_hours,apple_avg_listen_duration
0,E0,Chai Time Data Science Launch Announcement,NaN,NaN,NaN,NaN,NaN,NaN,Other,Masala Chai,2019-07-15,Evening,2019-07-21,157,https://www.youtube.com/watch?v=Ko_gxs42lM8,1,4433,86,1.94,45,131,3,82,4,0,2,3,https://anchor.fm/chaitimedatascience/episodes/Chai-Time-Data-Science-Launch-Announcement-e4mas9,0.0,553.0,491.0,262.0,359.0,29.0,1.0,117.0
1,E1,"Kaggle Triple Grandmaster, Abhishek Thakur Interview",Abhishek Thakur,Male,Norway,India,abhishek,abhi1thakur,Kaggle,Ginger Chai,2019-07-14,Evening,2019-07-22,2995,https://www.youtube.com/watch?v=Ezbo57Z33N8,0,25212,845,3.35,683,1528,142,335,55,0,5,60,https://anchor.fm/chaitimedatascience/episodes/Kaggle-Triple-Grandmaster--Abhishek-Thakur-Interview-e4mjoi,0.0,1271.0,826.0,608.0,456.0,56.0,25.0,1621.0
2,E2,"Interview with Kaggle Master, ML Engineer: Ryan Chesler",Ryan Chesler,Male,USA,USA,ryches,ryan_chesler,Kaggle,Masala Chai,2019-07-20,Afternoon,2019-07-26,2118,https://www.youtube.com/watch?v=SJVMSKig14k,0,3282,84,2.56,44,128,14,394,7,0,1,3,https://anchor.fm/chaitimedatascience/episodes/Interview-with-Kaggle-Master--ML-Engineer-Ryan-Chesler--Chai-Time-Data-Science-e4ntbt,0.0,681.0,398.0,274.0,214.0,19.0,10.0,1879.0
3,E3,"Interview with CEO of SharpestMinds, Edouard Harris",Edouard Harris,Male,Canada,Canada,NaN,neutronsNeurons,Industry,Kashmiri Kahwa,2019-07-23,Night,2019-07-29,3072,https://www.youtube.com/watch?v=69urmSt34Ac,0,2376,38,1.60,57,95,11,417,2,0,0,1,https://anchor.fm/chaitimedatascience/episodes/Interview-with-CEO-of-SharpestMinds--Edouard-Harris--Chai-Time-Data-Science-e4nti6,0.0,638.0,334.0,230.0,169.0,10.0,4.0,1344.0
4,E4,Data Science for Good: City of LA Kaggle Winning Solution Interview with Kaggle Kernels Grandmaster Shivam Bansal,Shivam Bansal,Male,Singapore,India,shivamb,shivamshaz,Kaggle,Apple Cinnamon,2019-07-14,Morning,2019-08-02,1048,https://www.youtube.com/watch?v=wMYX3KABHCk,0,3884,116,2.99,36,152,9,213,4,0,0,4,https://anchor.fm/chaitimedatascience/episodes/Data-Science-for-Good-City-of-LA-Kaggle-Winning-Solution-Interview-with-Kaggle-Kernels-Grandmaster-Shivam-Bansal-e4qc36,0.0,495.0,201.0,139.0,123.0,17.0,3.0,633.0


In [20]:
episodes["episode_id"].unique()

array(['E0', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10',
       'E11', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20',
       'E21', 'E22', 'E12', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28',
       'E29', 'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37',
       'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'M0', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'E46', 'E47', 'E48',
       'E49', 'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57',
       'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66',
       'E67', 'E68', 'E69', 'E70', 'E71', 'E72', 'E73', 'E74', 'E75'],
      dtype=object)

Couple of things of note here -

1. After `E11` we have `E13` and `E12` is after `E22`
2. We don't need the ones beginning with `M`

In [21]:
episodes = episodes[~episodes["episode_id"].str.startswith("M")]
episodes["episode_id"].unique()

array(['E0', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10',
       'E11', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20',
       'E21', 'E22', 'E12', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28',
       'E29', 'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37',
       'E38', 'E39', 'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46',
       'E47', 'E48', 'E49', 'E50', 'E51', 'E52', 'E53', 'E54', 'E55',
       'E56', 'E57', 'E58', 'E59', 'E60', 'E61', 'E62', 'E63', 'E64',
       'E65', 'E66', 'E67', 'E68', 'E69', 'E70', 'E71', 'E72', 'E73',
       'E74', 'E75'], dtype=object)

In [22]:
episodes[episodes["episode_id"]=="E12"]

,episode_id,episode_name,heroes,heroes_gender,heroes_location,heroes_nationality,heroes_kaggle_username,heroes_twitter_handle,category,flavour_of_tea,recording_date,recording_time,release_date,episode_duration,youtube_url,youtube_thumbnail_type,youtube_impressions,youtube_impression_views,youtube_ctr,youtube_nonimpression_views,youtube_views,youtube_watch_hours,youtube_avg_watch_duration,youtube_likes,youtube_dislikes,youtube_comments,youtube_subscribers,anchor_url,anchor_thumbnail_type,anchor_plays,spotify_starts,spotify_streams,spotify_listeners,apple_listeners,apple_listened_hours,apple_avg_listen_duration
22,E12,Freelancing in Machine Learning | Interview with Tuatini Godard,Tuatini Godard,Male,France,France,ekami66,NaN,Industry,Kashmiri Kahwa,2019-07-11,Morning,2019-10-29,2684,https://www.youtube.com/watch?v=AwJpKBMog6c,0,3659,61,1.67,53,114,17,537,4,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For some reason, reordering a single row seems to be a pain in Pandas...

In [27]:
episodes["episode_id_int"] = episodes["episode_id"].apply(lambda x:int(x.split("E")[1]))
episodes = episodes.sort_values(by=["episode_id_int"]).reset_index(drop=True)
episodes = episodes.drop(["episode_id_int"], axis=1)
episodes.iloc[[12, 22]]

,episode_id,episode_name,heroes,heroes_gender,heroes_location,heroes_nationality,heroes_kaggle_username,heroes_twitter_handle,category,flavour_of_tea,recording_date,recording_time,release_date,episode_duration,youtube_url,youtube_thumbnail_type,youtube_impressions,youtube_impression_views,youtube_ctr,youtube_nonimpression_views,youtube_views,youtube_watch_hours,youtube_avg_watch_duration,youtube_likes,youtube_dislikes,youtube_comments,youtube_subscribers,anchor_url,anchor_thumbnail_type,anchor_plays,spotify_starts,spotify_streams,spotify_listeners,apple_listeners,apple_listened_hours,apple_avg_listen_duration
12,E12,Freelancing in Machine Learning | Interview with Tuatini Godard,Tuatini Godard,Male,France,France,ekami66,NaN,Industry,Kashmiri Kahwa,2019-07-11,Morning,2019-10-29,2684,https://www.youtube.com/watch?v=AwJpKBMog6c,0,3659,61,1.67,53,114,17,537,4,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,E22,DeepMind & AlphaGo | Deep Learning Research | Swift For Tensorflow | Interview with Dr. Marc Lanctot,Marc Lactot,Male,Canada,Canada,NaN,sharky6000,Research,Apple Cinnamon,2019-08-27,Night,2019-10-27,4284,https://www.youtube.com/watch?v=-uUHR5VALB0,0,5959,168,2.82,476,644,66,369,19,0,1,17,https://anchor.fm/chaitimedatascience/episodes/DeepMind--AlphaGo--Deep-Learning-Research--Swift-For-Tensorflow--Interview-with-Dr--Marc-Lanctot-e7ks0s,0.0,693.0,177.0,119.0,110.0,42.0,28.0,2384.0


Some of the `youtube_url`s contain links to the Youtube playlist instead of the video itself. That messes up the download script a bit.

In [30]:
episodes["youtube_url"][65:]

65                                            https://www.youtube.com/watch?v=pQL892iT-dM
66                                            https://www.youtube.com/watch?v=VqysJmIqko8
67                                            https://www.youtube.com/watch?v=f5Qv3eSZpug
68                                            https://www.youtube.com/watch?v=ctss0hcD9SE
69                                            https://www.youtube.com/watch?v=hyJhwWshfbY
70                                            https://www.youtube.com/watch?v=n_IUOeiKwnE
71    https://www.youtube.com/watch?v=2dpaSTWdhSk&list=PLLvvXm0q8zUbiNdoIazGzlENMXvZ9bd3x
72    https://www.youtube.com/watch?v=iNZd_5T8tCI&list=PLLvvXm0q8zUbiNdoIazGzlENMXvZ9bd3x
73                                            https://www.youtube.com/watch?v=VeM1T7UaYTk
74                                            https://www.youtube.com/watch?v=aC9t9D7HpYE
75    https://www.youtube.com/watch?v=tq_XcFubgKo&list=PLLvvXm0q8zUbiNdoIazGzlENMXvZ9bd3x
Name: yout

Simpler to modify this manually

In [34]:
episodes.loc[71, "youtube_url"] = "https://www.youtube.com/watch?v=2dpaSTWdhSk"
episodes.loc[72, "youtube_url"] = "https://www.youtube.com/watch?v=iNZd_5T8tCI"
episodes.loc[75, "youtube_url"] = "https://www.youtube.com/watch?v=tq_XcFubgKo"
episodes["youtube_url"][70:]

70    https://www.youtube.com/watch?v=n_IUOeiKwnE
71    https://www.youtube.com/watch?v=2dpaSTWdhSk
72    https://www.youtube.com/watch?v=iNZd_5T8tCI
73    https://www.youtube.com/watch?v=VeM1T7UaYTk
74    https://www.youtube.com/watch?v=aC9t9D7HpYE
75    https://www.youtube.com/watch?v=tq_XcFubgKo
Name: youtube_url, dtype: object

One episode on youtube doesn't have the full audio. I am not removing them from here, but I will ignore them when extracting audio.

In [38]:
episodes[episodes["episode_id"] == "E46"]

,episode_id,episode_name,heroes,heroes_gender,heroes_location,heroes_nationality,heroes_kaggle_username,heroes_twitter_handle,category,flavour_of_tea,recording_date,recording_time,release_date,episode_duration,youtube_url,youtube_thumbnail_type,youtube_impressions,youtube_impression_views,youtube_ctr,youtube_nonimpression_views,youtube_views,youtube_watch_hours,youtube_avg_watch_duration,youtube_likes,youtube_dislikes,youtube_comments,youtube_subscribers,anchor_url,anchor_thumbnail_type,anchor_plays,spotify_starts,spotify_streams,spotify_listeners,apple_listeners,apple_listened_hours,apple_avg_listen_duration
46,E46,Classical Japanese Lit & ML | Kuzushiji recog kaggle comp | Interview with Tarin Clanuwat,Tarin Clanuwat,Female,Japan,Japan,NaN,tkasasagi,Research,Sulemani Chai,2020-02-03,Evening,2020-03-08,2162,https://www.youtube.com/watch?v=9E5JnTj8df0,1,4221,115,2.72,73,188,2,38,3,0,1,0,https://anchor.fm/chaitimedatascience/episodes/Interview-with-Tarin-Clanuwat--Classical-Japanese-Literature--ML--Kuzushiji-recognition-kaggle-comp-eb4rh6,3.0,605.0,43.0,29.0,33.0,17.0,7.0,1472.0


I think that cleans up "Episodes.csv" for my current requirements.

In [39]:
episodes.to_csv("/notebooks/storage/ctds_data/Episodes.csv")

## Cleaned Subtitles

Subtitles for E4 are missing. So, we will ignore that episode as well for now.

Someone noticed a potential issue with the rest of the subtitles - https://www.kaggle.com/rohanrao/chai-time-data-science/discussion/163343

Couple of lines were missing from the Subtitles. We will use their solution to this to clean up the dataset as well - https://www.kaggle.com/tomtillo/generating-cleaner-sublititles-file

In [84]:
filenames = []
path = "/notebooks/storage/ctds_data/Cleaned Subtitles/"
for file in os.listdir(path):
    filenames.append(path+file)
    
display(filenames[:3])

['/notebooks/storage/ctds_data/Cleaned Subtitles/E29.csv',
 '/notebooks/storage/ctds_data/Cleaned Subtitles/E32.csv',
 '/notebooks/storage/ctds_data/Cleaned Subtitles/E42.csv']

In [81]:
random_ep = filenames[random.randint(0, len(filenames)-1)]
print(random_ep[-7:])
ep_transcript = pd.read_csv(random_ep)
ep_transcript.head(2)

/E6.csv


,Time,Speaker,Text
0,0:13,Sanyam Bhutani,"Hey, this is Sanyam Bhutani and you're listening to ""Chai Time Data Science"", a podcast for data science enthusiast where I interview practitioners, researchers, and Kagglers about their journey, experience, and talk all things about data science.\n\nSanyam Bhutani 0:46 \nHi, everyone. Welcome to another episode of the ""Chai Time Data Science"" show. In this episode, I interview Shivam Bansal, who is one of the top kernel Grand Masters on Kaggle currently ranked fourth in the kernel rankings. Shivam is currently working as a research scientist at NUS and he has been working in the data science field for the past few years. We talk about his journey into data science, his Kaggle experience and about his recent competitions solution to the competition that he's recently won. We also talk about his pipeline for writing the amazing kernels, and blogposts that I'm definitely a fan of. Without further ado, enjoy the show.\n\nSanyam Bhutani 1:44 \nHello, Grand Master, thank you so much..."
1,1:51,Shivam Bansal,"Yeah, thank you so much Sanyam for inviting me. It's all also an honor for me to be part of this excellent ML series interview."


In [91]:
for file in filenames:
    
    df = pd.read_csv(file)
    if df["Time"][0] != "0:13":
        print(file[-7:])
        display(df.head(2))

E69.csv


,Time,Speaker,Text
0,0:00,Sanyam Bhutani,"Hello world, it's May 27, 2020. And I just turned 23. Welcome to ""Chai Time Data Science"", but a special AMA episode.\n\nSanyam Bhutani 0:23 \nSo this is sort of a special episode and I do apologise if you've come here for interviews with my machine learning heroes because this is slightly different. And this idea came out of actually a chai time chat at at work where I'm fortunate, really fortunate to be working with Rohan Rao and Parul Pandey who suggested this idea that for a birthday special, maybe I could invite questions from my heroes that I've been interviewing and just answer them. So this is what it is. To my surprise in the past three or four dayss that I've reached out to a lot of people, I got over 90 questions it might as well over 100 I didn't count exactly. And I also really wanted to do a celebration video my interview series is about to reach the number 100. Publicly, it's not 100 because I release to every week and the hundred is still to be released, but I tho..."
1,3:24,Sanyam Bhutani,"The next question is from Rohan Rao who's a data scientist at h2o Kaggle Grand Master and also worldwide known sodoku champion, what are the top enhancements or changes you're working on for CTDS? Anything to look forward to, in the near future? Uh funny story. The branding idea of CTDS actually came from Rohan, thanks to his suggestion. And yes, there are a lot of exciting things that I am I've been working on. So three things. I'm launching a new podcast. Yes, a new podcast called ""Chai Time Data Science News"" CTDS.news. You can find another write up that will live with this blog post. You can read all about it. But the idea is to give you a short news podcast, ideally in three to five minutes, always less than 10 minutes for you to be able to be on top of data science news, basically. Now if you know me, I am completely community driven. So if you have any thoughts around around that, please let me know the first episode is supposed to go live somewhere in the first week of June..."


E12.csv


,Time,Speaker,Text
0,0:00,Sanyam Bhutani,"Hello Tuatini. Thank you so much for taking the time to do this and joining the interview series for the second time. \n\nTuatini Godard 0:05 \nHi, Sanyam. Glad to be here.\n\nSanyam Bhutani 0:09 \nYeah. So today you're working as what people would think, to be the picture perfect life for any coder. You're travelling the world and workng on the move, which we'll just get to in a bit. Could you tell us how did you get interested in machine learning at first, like, what, what made you as you mentioned, quit your software desk job and shift to this field?"
1,0:35,Tuatini Godard,"Yeah, sure. So at first, what, four years ago, I was an Android developer. And I worked for a small startup. And the thing is, when I started working, work was really great. I'd said because I will start everything from scratch and add new feature and stuff like that. But at some point is started getting boring because you know, at some point, you will only start to do maintenance stuff. And everything start to get boring, basically. And so it got repetitive and very repetitive. Yeah. Because you only you're, you're only fixing bugs and, and maintaining stuff, basically. So at that point, I started to look for new opportunities. And the moment where I started to look for zero opportunities was exactly at the moment where deep learning started to boom. And I didn't know about that at all. Then I started to get interested in it. And when I started doing stuff on kaggle, and seeing how things works, I got immediately burnt and so I decided to;"


So, for most of the episodes, the subtitles are fine. We can ignore the first index for each of them for the audio clips.

For two of the episodes, the ones shown above, the result is different. For E69, we can again ignore the first row. But for E12, the timestamps are incorrect.

The 1st row's timestamp should start from about 01:54 based on a manual check. So, we will just correct that using a quick fix.

In [98]:
def add_time(row):
    m, s = row.split(":")
    s = int(s) + 54
    m = int(m) + 1
    if s > 60:
        s = s - 60
        m += 1
    return str(m) + ":" + str(s)

e12 = pd.read_csv('/notebooks/storage/ctds_data/Cleaned Subtitles/E12.csv')
e12["Time"] = e12["Time"].apply(add_time)
e12.head()

,Time,Speaker,Text
0,1:54,Sanyam Bhutani,"Hello Tuatini. Thank you so much for taking the time to do this and joining the interview series for the second time. \n\nTuatini Godard 0:05 \nHi, Sanyam. Glad to be here.\n\nSanyam Bhutani 0:09 \nYeah. So today you're working as what people would think, to be the picture perfect life for any coder. You're travelling the world and workng on the move, which we'll just get to in a bit. Could you tell us how did you get interested in machine learning at first, like, what, what made you as you mentioned, quit your software desk job and shift to this field?"
1,2:29,Tuatini Godard,"Yeah, sure. So at first, what, four years ago, I was an Android developer. And I worked for a small startup. And the thing is, when I started working, work was really great. I'd said because I will start everything from scratch and add new feature and stuff like that. But at some point is started getting boring because you know, at some point, you will only start to do maintenance stuff. And everything start to get boring, basically. And so it got repetitive and very repetitive. Yeah. Because you only you're, you're only fixing bugs and, and maintaining stuff, basically. So at that point, I started to look for new opportunities. And the moment where I started to look for zero opportunities was exactly at the moment where deep learning started to boom. And I didn't know about that at all. Then I started to get interested in it. And when I started doing stuff on kaggle, and seeing how things works, I got immediately burnt and so I decided to;"
2,3:44,Sanyam Bhutani,You got Kaggle bug.
3,3:46,Tuatini Godard,"Yeah, yeah. And, and then so I decided to tell my boss that I'm too that I wanted to quit to be on what we call on in France unemployment benefits. Where because when you get to quit a company, you can get unemployment benefits for two years in France if you work for two years makes it and so I was kind of like surviving on that stuff. And so I've learned that a sense from day one when I, when I was unemployed, and after one year in house, got into freelancing."
4,4:27,Sanyam Bhutani,"And to give the audience insight, I'll also have your profile linked but you have about 2200 hours of experience on on your profile on Upwork. So that really speaks for itself about your hard work. I also wanted to ask you about during this two years study plan of yours, you also you got started with freelancing towards the end. So what made you pick freelance side of work over what we call the job job the standard job?"


In [99]:
e12.to_csv('/notebooks/storage/ctds_data/Cleaned Subtitles/E12.csv')

All of the above might not really be the best possible fix because there might be other issues with the transcripts. But this should suffice for the time being.